# CrewAI with Custom LiteLLM Integration

This notebook demonstrates how to configure CrewAI to use a custom Language Model (LLM) endpoint via LiteLLM. We will set up the environment with your specific credentials, define agents and tasks, and run a crew to accomplish a goal.

## 1. Installation

First, let's install the necessary libraries. We need `crewai` for the agent framework and `crewai[tools]` for any specific tools the agents might use. The `openai` library is also required because LiteLLM exposes an OpenAI-compatible API, and we'll use the `ChatOpenAI` class from `langchain_openai` to interact with it.

In [ ]:
!pip install crewai crewai[tools] langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 10.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 54.5

## 2. Setup and LLM Configuration

This is the most critical step. We will import the required modules and configure the environment to use your LiteLLM server. Instead of letting CrewAI use a default LLM, we will instantiate our own `ChatOpenAI` object pointing to your custom endpoint.

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# --- Your LiteLLM Credentials ---
LITELLM_HOST = "llmproxy-gcp.go-yubi.in"
API_URL = f"https://{LITELLM_HOST}/v1"
API_KEY = "" # As provided in your example
MODEL_NAME = "openai/llama-3.3-70b-(US)" # As provided in your example

# Set environment variables for broader compatibility (optional but good practice)
os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = API_URL
os.environ["OPENAI_MODEL_NAME"] = MODEL_NAME

# --- Instantiate the Custom LLM ---
# We explicitly pass the base_url, model_name, and api_key to ensure
# it connects to your LiteLLM proxy, not the default OpenAI API.
lite_llm = ChatOpenAI(
    model=MODEL_NAME,
    base_url=API_URL,
    api_key=API_KEY
)

print("LLM Configuration Complete.")
print(f"Using model: {lite_llm.model_name}")
print(f"Targeting API Base: {lite_llm.openai_api_base}")

## 3. Define Agents

Now we'll define our agents. Each agent needs a `role`, `goal`, `backstory`, and importantly, the `llm` we just configured. We will create two agents:
1.  A **Market Researcher** to analyze the AI industry.
2.  A **Content Strategist** to write a blog post based on the research.

In [22]:
# Agent 1: Market Researcher
researcher = Agent(
  role='Senior Market Researcher',
  goal='Find and analyze the latest trends in the AI industry',
  backstory=(
    "You are a seasoned market researcher with a knack for identifying "
    "emerging trends. You use your analytical skills to provide concise "
    "and insightful reports on technology markets."
  ),
  verbose=True,
  allow_delegation=False,
  llm=lite_llm # Assign our custom LLM to this agent
)

# Agent 2: Content Strategist
writer = Agent(
  role='Expert Content Strategist',
  goal='Write a compelling and informative blog post about AI industry trends',
  backstory=(
    "You are a renowned content strategist, known for taking complex topics "
    "and turning them into engaging narratives. You craft content that is "
    "not only well-researched but also easy for a general audience to understand."
  ),
  verbose=True,
  allow_delegation=True,
  llm=lite_llm # Assign our custom LLM to this agent
)

print("Agents Defined.")

Agents Defined.


## 4. Create Tasks

Next, we define the tasks for our agents. The `writer`'s task will have a `context` dependency on the `researcher`'s task, ensuring they execute in the correct order.

In [23]:
# Task 1: Research
research_task = Task(
  description=(
    "Identify the top 3 most impactful trends in the AI industry for 2024. "
    "Focus on advancements in large language models, generative AI applications, "
    "and the enterprise adoption of AI technologies. "
    "Compile your findings into a concise summary."
  ),
  expected_output='A bullet-point list summarizing the top 3 AI trends.',
  agent=researcher
)

# Task 2: Writing
write_task = Task(
  description=(
    "Using the research findings, write a 500-word blog post titled 'The AI Revolution: Top 3 Trends to Watch in 2024'. "
    "The tone should be insightful yet accessible to a non-technical audience. "
    "Make sure to explain why each trend is significant."
  ),
  expected_output='A complete, well-structured blog post of approximately 500 words.',
  agent=writer,
  context=[research_task] # This task depends on the output of the research task
)

print("Tasks Defined.")

Tasks Defined.


## 5. Assemble and Run the Crew

Finally, we assemble the agents and tasks into a `Crew` and `kickoff()` the process. The crew will manage the execution flow, passing the output from the first task to the second.

In [24]:
# Create the crew
ai_trends_crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential, # Tasks will be executed one after another
  verbose=True # Provides detailed logs of the process
)

# Kick off the crew's work
print("\n--- Kicking off the Crew ---\n")
result = ai_trends_crew.kickoff()

print("\n--- Crew Execution Complete ---\n")
print("Final Result:")
print(result)


--- Kicking off the Crew ---



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 643033ad-97c6-402a-bc59-da7b603a2fe7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Market Researcher                                                                                │
│                                                                                                                 │
│  Task: Identify the top 3 most impactful trends in the AI industry for 2024. Focus on advancements in large     │
│  language models, generative AI applications, and the enterprise adoption of AI technologies. Compile your      │
│  findings into a concise summary.                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Market Researcher                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  * **Advancements in Large Language Models**: The development of more sophisticated large language models is    │
│  expected to continue in 2024, with a focus on improving their ability to understand and generate human-like    │
│  language. These models will have significant implications for applications such as chatbots, virtual           │
│  assistants, and content generation. As these models become more advanced, they will be able to perform tasks   │
│  such as language translation, text summarization, and question answering with greater accuracy and             │
│  efficiency.                                                                                                    │
│  * **Generative AI Applications**: Generative AI, which involves the use of AI algorithms to generate new       │
│  content such as images, videos, and music, is expected to be a major trend in 2024. This technology has the    │
│  potential to revolutionize industries such as entertainment, advertising, and education, and will likely be    │
│  used in a wide range of applications, from generating special effects in movies to creating personalized       │
│  educational content. As generative AI becomes more advanced, it will enable the creation of highly realistic   │
│  and customized content, which will have significant implications for the way we consume and interact with      │
│  media.                                                                                                         │
│  * **Enterprise Adoption of AI Technologies**: The adoption of AI technologies by enterprises is expected to    │
│  accelerate in 2024, as more companies recognize the potential of AI to improve efficiency, reduce costs, and   │
│  drive innovation. This trend will be driven by the development of more user-friendly and accessible AI tools,  │
│  as well as the growing availability of AI-powered cloud services. As enterprises adopt AI technologies, they   │
│  will be able to automate routine tasks, gain insights from large datasets, and make better decisions using     │
│  predictive analytics. This will have significant implications for industries such as healthcare, finance, and  │
│  retail, and will likely lead to the creation of new business models and revenue streams.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cb8540ce-a275-424a-bb3d-c622ca619b87                                                                     │
│  Agent: Senior Market Researcher                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The AI Revolution: Top 3 Trends to Watch in 2024                                                               │
│                                                                                                                 │
│  As we step into 2024, the world of Artificial Intelligence (AI) is poised for significant advancements that    │
│  will transform various aspects of our lives. From making AI more transparent and accountable to leveraging     │
│  its power for social good, the upcoming year promises to be exciting and impactful. In this blog post, we      │
│  will delve into the top 3 trends that are expected to dominate the AI landscape in 2024: Explainable AI, Edge  │
│  AI, and Ethical AI and AI for Social Good.                                                                     │
│                                                                                                                 │
│  **Trend 1: Explainable AI (XAI)**                                                                              │
│                                                                                                                 │
│  Explainable AI refers to the development of AI systems that can provide insights into their decision-making    │
│  processes. As AI becomes increasingly pervasive in our daily lives, there is a growing need to understand how  │
│  these systems arrive at their conclusions. XAI aims to address this issue by creating models that are          │
│  transparent, interpretable, and explainable. This trend has significant implications for industries such as    │
│  healthcare, finance, and law, where AI-driven decisions can have far-reaching consequences. By providing a     │
│  deeper understanding of AI decision-making, XAI can help build trust in AI systems and ensure that they are    │
│  fair, reliable, and unbiased.                                                                                  │
│                                                                                                                 │
│  **Trend 2: Edge AI**                                                                                           │
│                                                                                                                 │
│  Edge AI is a paradigm shift in the way AI is deployed and processed. Traditionally, AI models have been        │
│  trained and deployed in the cloud or on powerful servers. However, with the proliferation of IoT devices and   │
│  the need for real-time processing, Edge AI has emerged as a critical trend. Edge AI involves training and      │
│  deploying AI models on edge devices, such as smartphones, smart home devices, or autonomous vehicles. This     │
│  approach enables faster processing, reduced latency, and improved security. Edge AI has numerous               │
│  applications, including smart homes, industrial automation, and autonomous vehicles, where real-time           │
│  processing and decision-making are crucial.                                                                    │
│                                                                                                                 │
│  **Trend 3: Ethical AI and AI for Social Good**                                                                 │
│                                                                                                                 │
│  As AI becomes more ubiqu...                                                                                    │
│                                                                                                                 │
╰───────────────────────────────────────────────────────

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Content Strategist                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The AI Revolution: Top 3 Trends to Watch in 2024                                                               │
│                                                                                                                 │
│  As we step into 2024, the world of Artificial Intelligence (AI) is poised for significant advancements that    │
│  will transform various aspects of our lives. From making AI more transparent and accountable to leveraging     │
│  its power for social good, the upcoming year promises to be exciting and impactful. In this blog post, we      │
│  will delve into the top 3 trends that are expected to dominate the AI landscape in 2024: Explainable AI, Edge  │
│  AI, and Ethical AI and AI for Social Good.                                                                     │
│                                                                                                                 │
│  **Trend 1: Explainable AI (XAI)**                                                                              │
│                                                                                                                 │
│  Explainable AI refers to the development of AI systems that can provide insights into their decision-making    │
│  processes. As AI becomes increasingly pervasive in our daily lives, there is a growing need to understand how  │
│  these systems arrive at their conclusions. XAI aims to address this issue by creating models that are          │
│  transparent, interpretable, and explainable. This trend has significant implications for industries such as    │
│  healthcare, finance, and law, where AI-driven decisions can have far-reaching consequences. By providing a     │
│  deeper understanding of AI decision-making, XAI can help build trust in AI systems and ensure that they are    │
│  fair, reliable, and unbiased.                                                                                  │
│                                                                                                                 │
│  **Trend 2: Edge AI**                                                                                           │
│                                                                                                                 │
│  Edge AI is a paradigm shift in the way AI is deployed and processed. Traditionally, AI models have been        │
│  trained and deployed in the cloud or on powerful servers. However, with the proliferation of IoT devices and   │
│  the need for real-time processing, Edge AI has emerged as a critical trend. Edge AI involves training and      │
│  deploying AI models on edge devices, such as smartphones, smart home devices, or autonomous vehicles. This     │
│  approach enables faster processing, reduced latency, and improved security. Edge AI has numerous               │
│  applications, including smart homes, industrial automation, and autonomous vehicles, where real-time           │
│  processing and decision-making are crucial.                                                                    │
│                                                                                                                 │
│  **Trend 3: Ethical AI and AI for Social Good**                                                                 │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e462e9c1-01b7-413c-8a06-30d8dbe37831                                                                     │
│  Agent: Expert Content Strategist                                                                               │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 643033ad-97c6-402a-bc59-da7b603a2fe7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The AI Revolution: Top 3 Trends to Watch in 2024                                                 │
│                                                                                                                 │
│  As we step into 2024, the world of Artificial Intelligence (AI) is poised for significant advancements that    │
│  will transform various aspects of our lives. From making AI more transparent and accountable to leveraging     │
│  its power for social good, the upcoming year promises to be exciting and impactful. In this blog post, we      │
│  will delve into the top 3 trends that are expected to dominate the AI landscape in 2024: Explainable AI, Edge  │
│  AI, and Ethical AI and AI for Social Good.                                                                     │
│                                                                                                                 │
│  **Trend 1: Explainable AI (XAI)**                                                                              │
│                                                                                                                 │
│  Explainable AI refers to the development of AI systems that can provide insights into their decision-making    │
│  processes. As AI becomes increasingly pervasive in our daily lives, there is a growing need to understand how  │
│  these systems arrive at their conclusions. XAI aims to address this issue by creating models that are          │
│  transparent, interpretable, and explainable. This trend has significant implications for industries such as    │
│  healthcare, finance, and law, where AI-driven decisions can have far-reaching consequences. By providing a     │
│  deeper understanding of AI decision-making, XAI can help build trust in AI systems and ensure that they are    │
│  fair, reliable, and unbiased.                                                                                  │
│                                                                                                                 │
│  **Trend 2: Edge AI**                                                                                           │
│                                                                                                                 │
│  Edge AI is a paradigm shift in the way AI is deployed and processed. Traditionally, AI models have been        │
│  trained and deployed in the cloud or on powerful servers. However, with the proliferation of IoT devices and   │
│  the need for real-time processing, Edge AI has emerged as a critical trend. Edge AI involves training and      │
│  deploying AI models on edge devices, such as smartphones, smart home devices, or autonomous vehicles. This     │
│  approach enables faster processing, reduced latency, and improved security. Edge AI has numerous               │
│  applications, including smart homes, industrial automation, and autonomous vehicles, where real-time           │
│  processing and decision-making are crucial.                                                                    │
│                                                                                                                 │
│  **Trend 3: Ethical AI and AI for Social Good**       


--- Crew Execution Complete ---

Final Result:
The AI Revolution: Top 3 Trends to Watch in 2024

As we step into 2024, the world of Artificial Intelligence (AI) is poised for significant advancements that will transform various aspects of our lives. From making AI more transparent and accountable to leveraging its power for social good, the upcoming year promises to be exciting and impactful. In this blog post, we will delve into the top 3 trends that are expected to dominate the AI landscape in 2024: Explainable AI, Edge AI, and Ethical AI and AI for Social Good.

**Trend 1: Explainable AI (XAI)**

Explainable AI refers to the development of AI systems that can provide insights into their decision-making processes. As AI becomes increasingly pervasive in our daily lives, there is a growing need to understand how these systems arrive at their conclusions. XAI aims to address this issue by creating models that are transparent, interpretable, and explainable. This trend has significant 

## 6. Review the Output

The final output above is the result of the `write_task`, which is the last task in our sequential process. It should be a complete blog post, generated by your `openai/vertex-llama-api` model running on your LiteLLM server.